In [2]:
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()

In [4]:
import numpy as np
import pandas as pd

In [5]:
a = tf.add(3,5)
print(a)

Tensor("Add:0", shape=(), dtype=int32)


In [7]:
sess = tf.Session()

print(sess.run(a))

sess.close()

8


In [8]:
with tf.Session() as sess:
    print(sess.run(a))

8


In [18]:
x = 2
y = 3

op1 = tf.add(x,y)
op2 = tf.multiply(x,y)
useless = tf.multiply(x,op1)
op3 = tf.pow(op2,op1)

with tf.Session() as sess:
    op3, useless = sess.run([op3, useless])
    print(op3, useless)

7776 10
